# 자동차 환경 빅데이터 3차
- 과제2 : 건설기계 배출가스 관리 체게 구축
- 과제4 : 배출가스 전문정비제도 관리체계 구축
- 과제5 : 자동차 환경 빅데이터 제작사 공동 활용체게 구축

# Library

In [39]:
import pandas as pd
import os
import numpy as np
from datetime import datetime

In [6]:
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

In [200]:
# 경고문 없애는 코드
# UserWarning: Workbook contains no default style, apply openpyxl's default warn("Workbook contains no default style, apply openpyxl's default")
import warnings
warnings.simplefilter("ignore")

# Fold

In [536]:
raw_fold = 'D:/data/processing/big3/raw'
raw2_fold = 'D:/data/processing/big3/raw/BD2'
raw4_fold = 'D:/data/processing/big3/raw/BD4'
raw5_fold = 'D:/data/processing/big3/raw/BD5'
an_fold = 'D:/data/processing/big3/analysis'
an2_fold = 'D:/data/processing/big3/analysis/BD2'
an4_fold = 'D:/data/processing/big3/analysis/BD4'
an5_fold = 'D:/data/processing/big3/analysis/BD5'
df_fold = 'D:/data/processing/big3/df'
df2_fold = 'D:/data/processing/big3/df/BD2'
df4_fold = 'D:/data/processing/big3/df/BD4'
df5_fold = 'D:/data/processing/big3/df/BD5'
pub_fold = 'D:/data/processing/big3/raw/public'
folds = {'pub_fold':pub_fold, 'raw_fold':raw_fold, 'an_fold':an_fold, 'df_fold':df_fold, 
         'raw2_fold':raw2_fold, 'raw4_fold':raw4_fold, 'raw5_fold':raw5_fold, 
         'an2_fold:':an2_fold, 'an4_fold:':an4_fold, 'an5_fold:':an5_fold, 
         'df2_fold':df2_fold, 'df4_fold':df4_fold, 'df5_fold':df5_fold}
folds

{'pub_fold': 'D:/data/processing/big3/raw/public',
 'raw_fold': 'D:/data/processing/big3/raw',
 'an_fold': 'D:/data/processing/big3/analysis',
 'df_fold': 'D:/data/processing/big3/df',
 'raw2_fold': 'D:/data/processing/big3/raw/BD2',
 'raw4_fold': 'D:/data/processing/big3/raw/BD4',
 'raw5_fold': 'D:/data/processing/big3/raw/BD5',
 'an2_fold:': 'D:/data/processing/big3/analysis/BD2',
 'an4_fold:': 'D:/data/processing/big3/analysis/BD4',
 'an5_fold:': 'D:/data/processing/big3/analysis/BD5',
 'df2_fold': 'D:/data/processing/big3/df/BD2',
 'df4_fold': 'D:/data/processing/big3/df/BD4',
 'df5_fold': 'D:/data/processing/big3/df/BD5'}

# EDA
+ 과제2
    - 건설기계 인증자료
    - 건설기계 인증대장
+ 과제4
    - 자동차전문정비 : STD_TB_ERR_RES
    - 자동차전문정비상세 : STD_TB_ERR_RES_DET
+ 과제5
    - 구조변경검사 : STD_DLM_TB_ERP_GUBUN_BAECHUL

## 과제2 > 건설기계 인증자료(12개)

In [81]:
def construct_machine_certification_data_load(name, sheet_name=0):
    try:
        file_name = f'{name}.xlsx'
        cert_dt = pd.read_excel(os.path.join(raw2_fold + '/KENCIS 건설기계 인증자료_231018', file_name), sheet_name=sheet_name)
        return cert_dt
    except:
        file_name = f'{name}.xlsx'
        cert_dt = pd.read_excel(os.path.join(raw2_fold + '/KENCIS 건설기계 인증자료_231018', file_name))
        return cert_dt

### 정보 모음

In [93]:
cols = construct_machine_certification_data_load('KENCIS_건설기계인증정보_테이블명세서', '테이블정의서')
cols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   테이블명    12 non-null     object
 1   테이블설명   12 non-null     object
 2   컬럼명     649 non-null    object
 3   컬럼설명    644 non-null    object
 4   데이터형식   649 non-null    object
 5   길이      499 non-null    object
 6   Null    649 non-null    object
dtypes: object(7)
memory usage: 35.6+ KB


In [94]:
cols[['테이블명', '테이블설명']] = cols[['테이블명', '테이블설명']].ffill()

In [199]:
name_list = [
    'UNI_FC_TCMASTER', 
    'UNI_FC_TCERTIMODI', 
    'UNI_FC_ENG_TEFSPEC', 
    'UNI_FC_TEEMISSN', 
    'UNI_FC_TEEMISSN_ENG_RES', 
    'UNI_FC_TESTINFO', 
    'UNI_FC_ENG_LIST', 
    'UNI_FC_ENDUENG_EMI', 
    'UNI_FC_ENDUENGTEST', 
    'UNI_FC_ENDUENGTEST_RES', 
    'UNI_FC_KIENGTEST', 
    'UNI_FC_KIENGTEST_RES', 
    ]
len(name_list)

12

In [110]:
cols_dict = {}
for t in name_list:
    cols_dict[t] = {a:b for a, b in cols.loc[cols['테이블명'] == t, ['컬럼명', '컬럼설명']].values}

In [111]:
cols_dict

{'UNI_FC_ENDUENG_EMI': {'ADM_NO': '관리번호',
  'CERTI_MODI_SNO': '변경인증일련번호',
  'CO_GRAD': 'CO기울기',
  'CO_YITCT': 'CO Y절편',
  'MOD_DATETIME': '수정일시',
  'MOD_ID': '수정아이디',
  'NMHC_GRAD': 'NMHC기울기',
  'NMHC_YITCT': 'NMHC Y절편',
  'NOX_GRAD': 'NOX기울기',
  'NOX_YITCT': 'NOX Y절편',
  'PM_GRAD': 'PM기울기',
  'PM_YITCT': 'PM Y절편',
  'REG_DATETIME': '등록일시',
  'REG_ID': '등록아이디',
  'TEST_METHOD_CHK': '시험방법체크값',
  'TEST_MOD': '배출가스시험모드',
  'TEST_RST_SNO': '시험결과일련번호',
  'PN_GRAD': 'PN 기울기',
  'PN_YITCT': 'PN Y절편'},
 'UNI_FC_ENDUENGTEST': {'ADM_NO': '관리번호',
  'CERTI_MODI_SNO': '변경인증일련번호',
  'ENG_ID_NO': '대표원동기명(원동기코드)',
  'ENG_NO': '원동기 번호',
  'ENG_TYPE': '엔진형식',
  'MOD_DATETIME': '수정일시',
  'MOD_ID': '수정아이디',
  'REG_DATETIME': '등록일시',
  'REG_ID': '등록아이디',
  'TEST_DATE': '시험일',
  'TEST_ENG_SNO': '시험원동기일련번호',
  'TEST_NO': '시험번호'},
 'UNI_FC_ENDUENGTEST_RES': {'ADM_NO': '관리번호',
  'CERTI_MODI_SNO': '인증 일련번호',
  'P_ENG_YN': '내구대표차원동기여부',
  'SEL_ADM_NO': '내구 동일원동기  관리번호',
  'TEST_METHOD_CHK': '시험방법체크값',
  'TEST_MO

In [192]:
name = 'UNI_FC_KIENGTEST_RES'
ct01 = construct_machine_certification_data_load(name)
ct01.info()

c:\Users\kbjung\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2740 entries, 0 to 2739
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ADM_NO            2740 non-null   int64  
 1   CERTI_MODI_SNO    2740 non-null   int64  
 2   P_ENG_YN          2704 non-null   float64
 3   SEL_ADM_NO        30 non-null     float64
 4   TEST_METHOD_CHK   2740 non-null   object 
 5   TEST_MOD          2740 non-null   int64  
 6   TEST_RST_SNO      2740 non-null   int64  
 7   TEST_RST_SNO_SEQ  2740 non-null   int64  
 8   FUEL              0 non-null      float64
 9   CO                953 non-null    float64
 10  NMHC              924 non-null    float64
 11  NOX               943 non-null    float64
 12  PM                943 non-null    float64
 13  REG_ID            2740 non-null   object 
 14  REG_DATETIME      2740 non-null   object 
 15  MOD_ID            162 non-null    object 
 16  MOD_DATETIME      162 non-null    object 


In [187]:
cols_dict[name]

{'ADM_NO': '관리번호',
 'CERTI_MODI_SNO': '변경인증일련번호',
 'EF_SNO': '동일차종일련번호',
 'ENG_DISPL': '배기량',
 'ENG_ID_NO': '대표원동기명(원동기코드)',
 'ENG_SNO': '원동기장착일련번호',
 'ENG_TYPE': '엔진형식',
 'ETC': '기타 정보',
 'MAX_PWR_KW': '최대출력kW',
 'MAX_PWR_RPM': '최대출력 회전수',
 'MOD_DATETIME': '수정일시',
 'MOD_ID': '수정아이디',
 'MODI_REF_EF_SNO': '변경부모동일원동기번호',
 'REG_DATETIME': '등록일시',
 'REG_ID': '등록아이디',
 'VEH_IN_COM': '수입사',
 'VEH_MAKER': '제작사',
 'VEH_NM': '차명',
 'VEH_PRE_NM': '건설(농)기계 수입사 대표자 성명',
 'VEH_TYPE': '자동차형식',
 'VEH_TYPE_YN': '인증서 표출 여부'}

In [190]:
cols_df = pd.DataFrame({'컬럼명':cols_dict[name].keys(), '컬럼설명':cols_dict[name].values()})
cols_df

,컬럼명,컬럼설명
0,ADM_NO,관리번호
1,CERTI_MODI_SNO,변경인증일련번호
2,EF_SNO,동일차종일련번호
3,ENG_DISPL,배기량
4,ENG_ID_NO,대표원동기명(원동기코드)
5,ENG_SNO,원동기장착일련번호
6,ENG_TYPE,엔진형식
7,ETC,기타 정보
8,MAX_PWR_KW,최대출력kW
9,MAX_PWR_RPM,최대출력 회전수


In [191]:
temp = ct01.isnull().sum().reset_index().rename(columns={'index':'컬럼명', 0:'빈 값 개수'})
temp['총 행수'] = ct01.shape[0]
temp.insert(0, '번호', [x for x in range(1, temp.shape[0] + 1)])
ttemp = temp.merge(cols_df, on='컬럼명', how='left')
ttemp = ttemp[['번호', '컬럼명', '컬럼설명', '빈 값 개수', '총 행수']]
ttemp

,번호,컬럼명,컬럼설명,빈 값 개수,총 행수
0,1,ENG_SNO,원동기장착일련번호,0,9521
1,2,ADM_NO,관리번호,0,9521
2,3,CERTI_MODI_SNO,변경인증일련번호,0,9521
3,4,EF_SNO,동일차종일련번호,0,9521
4,5,ENG_TYPE,엔진형식,9516,9521
5,6,ENG_ID_NO,대표원동기명(원동기코드),9516,9521
6,7,MAX_PWR_KW,최대출력kW,9516,9521
7,8,MAX_PWR_RPM,최대출력 회전수,9516,9521
8,9,ENG_DISPL,배기량,9516,9521
9,10,VEH_IN_COM,수입사,240,9521


In [195]:
today_date = datetime.today().strftime('%Y.%m.%d')
today_date

'2023.10.31'

#### 컬럼, 데이터 정보들 출력

In [201]:
# 41.2s
with pd.ExcelWriter(os.path.join(an2_fold, f'건설기계_인증자료_데이터정보({today_date}).xlsx')) as writer:
    for i, name in enumerate(name_list):
        cols_df = pd.DataFrame({'컬럼명':cols_dict[name].keys(), '컬럼설명':cols_dict[name].values()})
        ct = construct_machine_certification_data_load(name)
        temp = ct.isnull().sum().reset_index().rename(columns={'index':'컬럼명', 0:'빈 값 개수'})
        temp['총 행수'] = ct.shape[0]
        temp.insert(0, '번호', [x for x in range(1, temp.shape[0] + 1)])
        ttemp = temp.merge(cols_df, on='컬럼명', how='left')
        ttemp = ttemp[['번호', '컬럼명', '컬럼설명', '빈 값 개수', '총 행수']]
        ttemp.to_excel(writer, sheet_name=f'{i+1}. {name}', index=False)

In [202]:
name_list

['UNI_FC_TCMASTER',
 'UNI_FC_TCERTIMODI',
 'UNI_FC_ENG_TEFSPEC',
 'UNI_FC_TEEMISSN',
 'UNI_FC_TEEMISSN_ENG_RES',
 'UNI_FC_TESTINFO',
 'UNI_FC_ENG_LIST',
 'UNI_FC_ENDUENG_EMI',
 'UNI_FC_ENDUENGTEST',
 'UNI_FC_ENDUENGTEST_RES',
 'UNI_FC_KIENGTEST',
 'UNI_FC_KIENGTEST_RES']

### 1\. UNI_FC_TCMASTER

In [592]:
name01 = name_list[0]
name01

'UNI_FC_TCMASTER'

In [593]:
# 1.3s
ct01 = construct_machine_certification_data_load(name01)
ct01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Columns: 159 entries, ADM_NO to NOISE_STD_NOTE
dtypes: datetime64[ns](1), float64(127), int64(2), object(29)
memory usage: 775.3+ KB


In [594]:
# CERTI_DIV(인증구분) : 1, 2
# EMIS_CERTI_NO(배출가스인증번호) : 00EN*AA*00 형식(숫자, 영문, 특수문자 혼합 10자리)
# EMIS_VEH_CLASS(배출가스 차종) : CE
# CERTI_STATE 처분내용 : nan, '90', 'C0', '10', '42', '46', '41'
# RECEIVE_STATE 반영 전 데이타 : nan,  0.
# NATION 제작국가 : nan, 'SW', 'US', 'KR', 'JP', 'GM', 'IT', 'FN', 'UK', 'CN', 'FR', 'SP', 'SU', 'ID'
# SELF_TEST 시설확인유무 : nan, 'Y', 'N'
# EMIS_CERTI_SNO 배출가스인증번호 : nan,  1.,  2.,  7.,  3.,  5., 12., 13.,  4.,  6., 10.,  8., 11., 9.
# OFFICE_CD 업체코드 : 'DDDC', 'VVIC', 'HIIC', 'KJDC', 'SSIC', 'DSDC', 'HDDC', 'WYIC', 'SJIC', 'ASIC', 'DPIC', 'LSIC', 'KBIC', 'CMIC', 'WTIC', 'MEIC', 'WJIC', 'HCDC', 'TMIC', 'LHIC', 'MXIC', 'HWDC', 'SCIC', 'EXAN', 'RADC', 'TYDC'
# OLDCERTI 과거자료여부 : nan, 'Y'
# REG2_YN 재신청여부 : nan, 'N'
# SNOTE 취소반납사유 : nan, '권리 및 의무 승계', '권리 및 의무 승계 신고', '권리 및 의무 신고에 따른 인증서 재교부'
# EF_CNT 동일차량 대수 : nan,  5.,  2.,  1.,  6.,  4.,  3., 22.,  9.,  0., 31.,  8., 14., 23.,  7., 15., 16., 11., 51., 13., 17., 10., 18.
# ELEC_CAR_GB 전기차 구분 ( N:일반,E:전기차(사용안함),H:하이브리드(사용안함), C:친환경차 ) : nan, 'N'
# CONF_ISSUE_NO 발급문서번호 : nan,  821.,  941.,  901., 1141., 1022., 1122.,  822.
# APPROVAL_YN 결재여부 : 'N', 'Y'
# ENG_CAR_GB 건설기계/농업기계구분 : nan, 'C', 'A', 'CA'

In [597]:
col01 = 'CERTI_DIV'
print(cols_dict[name01][col01])
set(ct01[col01].unique())

인증구분


{1, 2}

### 2\. UNI_FC_TCERTIMODI

In [598]:
name02 = name_list[1]
name02

'UNI_FC_TCERTIMODI'

In [599]:
# 5.0s
ct02 = construct_machine_certification_data_load(name02)
ct02.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830 entries, 0 to 2829
Data columns (total 91 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   CERTI_MODI_SNO        2830 non-null   int64         
 1   REG_DATETIME          2830 non-null   object        
 2   CERTI_MODI_DIV        2375 non-null   float64       
 3   EMIS_NOISE_DIV        2670 non-null   float64       
 4   EMIS_MODI_ITEM        0 non-null      float64       
 5   NOISE_MODI_ITEM       0 non-null      float64       
 6   EMIS_MODI_REASON      2346 non-null   object        
 7   NOISE_MODI_REASON     0 non-null      float64       
 8   MODI_ITEM_ETC         0 non-null      float64       
 9   RECT_DATE             2828 non-null   object        
 10  APPLI_ID              2670 non-null   object        
 11  OFFICER_ID            0 non-null      float64       
 12  MODI_EXPLAIN          0 non-null      float64       
 13  EMIS_CHG_EXAM     

In [600]:
# CERTI_MODI_SNO 변경인증일련번호 : 0 ~ 77
# CERTI_MODI_DIV 변경구분(1:변경인증, 2:변경보고) : 1.,  2., nan
# EMIS_NOISE_DIV 배출소음구분(1:배출 2:소음 3:배출+소음) : 1., nan
# MODIF_PROC_STATE 변경인증 진행상태 : nan, 0.0
# CERTI_STATE 인증진행상태 : nan, '10', '20', '30', '42', '46', '90', 'C', 'C0', 'C2'
# OLDCERTI 과거자료여부 : nan, 'N', 'Y'
# CERTI_SNO TCERTIMODI의IDX : nan, 1.0 ~ 48.0
# EMIS_DOC 배출가스인증서교부 공문 내용 : 1.0, nan
# CERTI_DIV 인증구분(1:국내,2:수입) : 1.0, 2.0, nan
# APPROVAL_YN 결재여부 : 'N', 'Y'
# MAKE_MTRL_NO 인증신청 작성자료번호 : 1.0, 3.0, 4.0, 5.0, 6.0, nan
# STANDARD_SLT 배출허용기준 : nan, 1.0, 2.0, 3.0, 4.0, 6.0, 7.0, 8.0, 9.0, 10.0
# STANDARD_SLT2 배출허용기준2 : nan, 1.0, 2.0, 3.0, 4.0, 6.0, 7.0, 8.0, 9.0, 10.0
# EMIS_ROWS_CODE 배출가스저감기술 : nan, '1', '1/2', '1/2/3', '1/2/3/4', '1/2/3/4/5', '1/2/4', '1/2/4/5', '1/3', '1/3/4', '1/3/4/5', '1/4', '1/4/5', '1/5', '2', '2/3/4', '2/4', '3/4', '5'
# EMIS_LOWS 배출가스저감기술코드 : nan, 'In-cylinder 방식', 'LNC', 'Turbocharger, Common rail', 'VGT, common rail', '연료장치, 터보챠저', '전자제어 연료분사 시스템', '전자제어 연료분사시스템', '전자제어장치,커먼레일', '커먼레일', '터보', '터보, 기계식', '터보차저', '터보차져'

In [601]:
col02 = 'CERTI_MODI_DIV'
print(cols_dict[name02][col02])
set(ct02[col02].unique())

변경구분(1:변경인증, 2:변경보고)


{1.0, 2.0, nan}

### 3\. UNI_FC_ENG_TEFSPEC

In [602]:
name03 = name_list[2]
name03

'UNI_FC_ENG_TEFSPEC'

In [603]:
# 16.1s
ct03 = construct_machine_certification_data_load(name03)
ct03.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8646 entries, 0 to 8645
Columns: 108 entries, ADM_NO to MAX_OUTT_POWER_UNIT
dtypes: float64(23), int64(3), object(82)
memory usage: 7.1+ MB


In [604]:
# CERTI_MODI_SNO 변경인증일련번호 : 0 ~ 77
# EF_SNO 동일원동기일련번호 : 0 ~ 256
# CYCLE 싸이클(행정) : nan, '4', '4cycle', '4사이클', '4행정', '5'
# COMB_TYPE 연소방식 : nan, 1.0, 2.0
# COOL_TYPE 냉각매체 : nan, 1.0
# CAT_TYPE 종류 : nan, 1.0, 2.0, 8.0, 9.0, 12.0, 15.0, 19.0, 20.0, 21.0, 22.0, 24.0, 99.0
# ENG_CAR_GB 건설기계/농업기계구분 : nan, 'A', 'C', 'CA'

In [605]:
col03 = 'ENG_CAR_GB'
print(cols_dict[name03][col03])
set(ct03[col03].unique())

건설기계/농업기계구분


{'A', 'C', 'CA', nan}

### 4\. UNI_FC_TEEMISSN

In [606]:
name04 = name_list[3]
name04

'UNI_FC_TEEMISSN'

In [607]:
# 0.3s
ct04 = construct_machine_certification_data_load(name04)
ct04.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1058 entries, 0 to 1057
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ADM_NO           1058 non-null   int64  
 1   CERTI_MODI_SNO   1058 non-null   int64  
 2   FUEL             882 non-null    object 
 3   SEL_STAND        592 non-null    float64
 4   F_STAND          609 non-null    float64
 5   GU_TERM          577 non-null    float64
 6   TEST_DIV         0 non-null      float64
 7   TEST_METHOD_CHK  1053 non-null   object 
 8   COL              0 non-null      float64
 9   REG_ID           1058 non-null   object 
 10  REG_DATETIME     1058 non-null   object 
 11  MOD_ID           538 non-null    object 
 12  MOD_DATETIME     538 non-null    object 
 13  TBL_PK_SEQ_ID    1058 non-null   int64  
 14  LEV_CRTR_CD      0 non-null      float64
dtypes: float64(6), int64(3), object(6)
memory usage: 124.1+ KB


In [608]:
# CERTI_MODI_SNO 인증일련번호 : 0 ~ 70(빈 번호 존재)
# FUEL 사용연료코드 : nan, ‘D’
# SEL_STAND 적용기준 : nan, 3.0, 5.0, 6.0, 7.0, 8.0
# F_STAND 외국기준 : nan, 5.0, 6.0, 9.0, 10.0, 11.0

In [609]:
col04 = 'LEV_CRTR_CD'
print(cols_dict[name04][col04])
set(ct04[col04].unique())

저배출차량 기준 코드


{nan}

### 5\. UNI_FC_TEEMISSN_ENG_RES

In [610]:
name05 = name_list[4]
name05

'UNI_FC_TEEMISSN_ENG_RES'

In [611]:
# 6.4s
ct05 = construct_machine_certification_data_load(name05)
ct05.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2761 entries, 0 to 2760
Columns: 101 entries, ADM_NO to NH3_ST_PERMSS
dtypes: float64(77), int64(5), object(19)
memory usage: 2.1+ MB


In [ ]:
# CERTI_MODI_SNO 인증일련번호 : 0 ~ 35(빈 번호 존재)
# TEST_RST_CODE 시험결과구분코드(차량별시험결과=00,종합결과=99) : 0, 99
# TEST_MOD 시험모드(CVS-75,Highway,US06,SC03,Cold CO...) : 1, 2, 3, 99

In [614]:
col05 = 'TEST_MOD'
print(cols_dict[name05][col05])
set(ct05[col05].unique())

시험모드(CVS-75,Highway,US06,SC03,Cold CO...)


{1, 2, 3, 99}

### 6\. UNI_FC_TESTINFO

In [615]:
name06 = name_list[5]
name06

'UNI_FC_TESTINFO'

In [616]:
# 1.3s
ct06 = construct_machine_certification_data_load(name06)
ct06.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1044 entries, 0 to 1043
Data columns (total 67 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ADM_NO                         1044 non-null   int64  
 1   EF_SNO                         0 non-null      float64
 2   CERTI_MODI_SNO                 1044 non-null   int64  
 3   INFO1                          177 non-null    object 
 4   INFO2                          1044 non-null   object 
 5   INFO3                          0 non-null      float64
 6   TC_TYPE                        1044 non-null   int64  
 7   INFO4                          0 non-null      float64
 8   ELEC_INFO1                     0 non-null      float64
 9   ELEC_TECH                      0 non-null      float64
 10  ELEC_INFO1_TXT                 0 non-null      float64
 11  ELEC_WARR_ACCUMU               0 non-null      float64
 12  ELEC_WARR_CHARGER              0 non-null      f

In [617]:
# CERTI_MODI_SNO 인증일련번호 : 0 ~ 70(빈 번호 존재)
# TC_TYPE 적용기준(기본/변경) : 1

In [618]:
col06 = 'TC_TYPE'
print(cols_dict[name06][col06])
set(ct06[col06].unique())

적용기준(기본/변경)


{1}

### 7\. UNI_FC_ENG_LIST

In [619]:
name07 = name_list[6]
name07

'UNI_FC_ENG_LIST'

In [620]:
# 3.7s
ct07 = construct_machine_certification_data_load(name07)
ct07.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9521 entries, 0 to 9520
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ENG_SNO          9521 non-null   int64  
 1   ADM_NO           9521 non-null   int64  
 2   CERTI_MODI_SNO   9521 non-null   int64  
 3   EF_SNO           9521 non-null   int64  
 4   ENG_TYPE         5 non-null      object 
 5   ENG_ID_NO        5 non-null      object 
 6   MAX_PWR_KW       5 non-null      float64
 7   MAX_PWR_RPM      5 non-null      float64
 8   ENG_DISPL        5 non-null      float64
 9   VEH_IN_COM       9281 non-null   object 
 10  VEH_PRE_NM       9279 non-null   object 
 11  VEH_NM           9287 non-null   object 
 12  VEH_MAKER        9291 non-null   object 
 13  VEH_TYPE         9290 non-null   object 
 14  ETC              58 non-null     object 
 15  REG_DATETIME     9521 non-null   object 
 16  REG_ID           2986 non-null   object 
 17  MOD_DATETIME  

In [621]:
# ENG_SNO 원동기장착일련번호 : 0 ~ 21
# CERTI_MODI_SNO 변경인증일련번호 : 0 ~ 77
# EF_SNO 동일차종일련번호 : 0 ~ 256
# ENG_ID_NO 대표원동기명(원동기코드) : nan, 'CFYI520/CFYI520U', 'F5GFL414J*B', 'F5HFL464A*F',
# VEH_TYPE_YN 인증서 표출 여부 : nan, 'Y'

In [622]:
col07 = 'VEH_TYPE_YN'
print(cols_dict[name07][col07])
set(ct07[col07].unique())

인증서 표출 여부


{'Y', nan}

### 8\. UNI_FC_ENDUENG_EMI

In [623]:
name08 = name_list[7]
name08

'UNI_FC_ENDUENG_EMI'

In [624]:
# 0.2s
ct08 = construct_machine_certification_data_load(name08)
ct08.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411 entries, 0 to 410
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ADM_NO           411 non-null    int64  
 1   CERTI_MODI_SNO   411 non-null    int64  
 2   TEST_METHOD_CHK  411 non-null    object 
 3   TEST_MOD         411 non-null    int64  
 4   TEST_RST_SNO     411 non-null    int64  
 5   NOX_GRAD         167 non-null    float64
 6   NMHC_GRAD        153 non-null    float64
 7   CO_GRAD          163 non-null    float64
 8   PM_GRAD          166 non-null    float64
 9   NOX_YITCT        166 non-null    float64
 10  NMHC_YITCT       153 non-null    float64
 11  CO_YITCT         165 non-null    float64
 12  PM_YITCT         165 non-null    float64
 13  REG_ID           411 non-null    object 
 14  REG_DATETIME     375 non-null    object 
 15  MOD_ID           57 non-null     object 
 16  MOD_DATETIME     21 non-null     object 
 17  PN_GRAD         

In [625]:
# CERTI_MODI_SNO 변경인증일련번호 : 0 ~ 22(빈 번호 존재)
# TEST_METHOD_CHK 시험방법체크값 : 'NNNN', 'NNNY', 'NNYN', 'NNYY', 'NYNN', 'NYYN', 'YNNN', 'YNYN', 'YYYY'
# TEST_MOD 배출가스시험모드 : 1, 3, 4
# TEST_RST_SNO 시험결과일련번호 : 1

In [627]:
col08 = 'TEST_RST_SNO'
print(cols_dict[name08][col08])
set(ct08[col08].unique())

시험결과일련번호


{1}

### 9\. UNI_FC_ENDUENGTEST

In [628]:
name09 = name_list[8]
name09

'UNI_FC_ENDUENGTEST'

In [629]:
# 0.1s
ct09 = construct_machine_certification_data_load(name09)
ct09.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ADM_NO          430 non-null    int64 
 1   CERTI_MODI_SNO  430 non-null    int64 
 2   TEST_ENG_SNO    430 non-null    int64 
 3   ENG_TYPE        334 non-null    object
 4   ENG_ID_NO       315 non-null    object
 5   ENG_NO          319 non-null    object
 6   TEST_NO         277 non-null    object
 7   TEST_DATE       318 non-null    object
 8   REG_ID          407 non-null    object
 9   REG_DATETIME    430 non-null    object
 10  MOD_ID          56 non-null     object
 11  MOD_DATETIME    51 non-null     object
dtypes: int64(3), object(9)
memory usage: 40.4+ KB


In [630]:
# CERTI_MODI_SNO 변경인증일련번호 : 0 ~ 22(빈 번호 존재)
# TEST_ENG_SNO 시험원동기일련번호 : 0 ~ 19

In [632]:
col09 = 'TEST_ENG_SNO'
print(cols_dict[name09][col09])
set(ct09[col09].unique())

시험원동기일련번호


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}

### 10\. UNI_FC_ENDUENGTEST_RES

In [635]:
name10 = name_list[9]
name10

'UNI_FC_ENDUENGTEST_RES'

In [636]:
# 4.2s
ct10 = construct_machine_certification_data_load(name10)
ct10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7581 entries, 0 to 7580
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ADM_NO            7581 non-null   int64  
 1   CERTI_MODI_SNO    7581 non-null   int64  
 2   P_ENG_YN          7581 non-null   int64  
 3   SEL_ADM_NO        4236 non-null   float64
 4   TEST_METHOD_CHK   7581 non-null   object 
 5   TEST_MOD          7581 non-null   int64  
 6   TEST_RST_SNO      7581 non-null   int64  
 7   TEST_RST_SNO_SEQ  7581 non-null   int64  
 8   FUEL              0 non-null      float64
 9   DDC_USE_YN        6724 non-null   float64
 10  TEST_CNT          0 non-null      float64
 11  TEST_TIME         7100 non-null   float64
 12  TEST_PLACE        7054 non-null   object 
 13  TEST_DATE         7025 non-null   object 
 14  CO                6034 non-null   float64
 15  NOX               5960 non-null   float64
 16  PM                5901 non-null   float64


In [637]:
# CERTI_MODI_SNO 인증 일련번호 : 0 ~ 22(빈 번호 존재)
# P_ENG_YN 내구대표차원동기여부 : 0, 1, 2
# TEST_METHOD_CHK 시험방법체크값 : 'NNNN', 'NNNY', 'NNYN', 'NNYY', 'NYNN', 'NYYN', 'YNNN', 'YNYN', 'YYYY'
# TEST_MOD 시험모드(CVS-75,Highway,US06,SC03,Cold CO...) : 1, 2
# TEST_RST_SNO 시험결과일련번호 : 1 ~ 268
# TEST_RST_SNO_SEQ 시험결과일련번호의항목순번 : 1 ~ 4
# DDC_USE_YN 지정열화계수 사용유무(사용:1,미사용:0) : nan, 1.0, 2.0

In [638]:
col10 = 'TEST_CNT'
print(cols_dict[name10][col10])
set(ct10[col10].unique())

회차


{nan}

### 11\. UNI_FC_KIENGTEST

In [639]:
name11 = name_list[10]
name11

'UNI_FC_KIENGTEST'

In [640]:
# 0.1s
ct11 = construct_machine_certification_data_load(name11)
ct11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307 entries, 0 to 306
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ADM_NO          307 non-null    int64 
 1   CERTI_MODI_SNO  307 non-null    int64 
 2   TEST_ENG_SNO    307 non-null    int64 
 3   ENG_TYPE        209 non-null    object
 4   ENG_ID_NO       196 non-null    object
 5   ENG_NO          209 non-null    object
 6   TEST_NO         179 non-null    object
 7   TEST_DATE       203 non-null    object
 8   REG_ID          307 non-null    object
 9   REG_DATETIME    307 non-null    object
 10  MOD_ID          29 non-null     object
 11  MOD_DATETIME    29 non-null     object
dtypes: int64(3), object(9)
memory usage: 28.9+ KB


In [641]:
# CERTI_MODI_SNO 변경인증일련번호 : 0 ~ 35(빈 번호 존재)
# TEST_ENG_SNO 시험원동기일련번호 : 0 ~ 5

In [642]:
col11 = 'TEST_ENG_SNO'
print(cols_dict[name11][col11])
set(ct11[col11].unique())

시험원동기일련번호


{0, 1, 2, 3, 4, 5}

### 12\. UNI_FC_KIENGTEST_RES

In [643]:
name12 = name_list[11]
name12

'UNI_FC_KIENGTEST_RES'

In [644]:
# 1.3s
ct12 = construct_machine_certification_data_load(name12)
ct12.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2740 entries, 0 to 2739
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ADM_NO            2740 non-null   int64  
 1   CERTI_MODI_SNO    2740 non-null   int64  
 2   P_ENG_YN          2704 non-null   float64
 3   SEL_ADM_NO        30 non-null     float64
 4   TEST_METHOD_CHK   2740 non-null   object 
 5   TEST_MOD          2740 non-null   int64  
 6   TEST_RST_SNO      2740 non-null   int64  
 7   TEST_RST_SNO_SEQ  2740 non-null   int64  
 8   FUEL              0 non-null      float64
 9   CO                953 non-null    float64
 10  NMHC              924 non-null    float64
 11  NOX               943 non-null    float64
 12  PM                943 non-null    float64
 13  REG_ID            2740 non-null   object 
 14  REG_DATETIME      2740 non-null   object 
 15  MOD_ID            162 non-null    object 
 16  MOD_DATETIME      162 non-null    object 


In [645]:
# CERTI_MODI_SNO 인증 일련번호 : 0 ~ 35(빈 번호 존재)
# P_ENG_YN KI대표원동기 여부 : nan, 1.0, 2.0
# TEST_METHOD_CHK 시험방법체크값 : 'NNNN', 'NNYN', 'NYNN'
# TEST_MOD 시험모드(CVS-75,Highway,US06,SC03,Cold CO...) : 1, 2
# TEST_RST_SNO 시험결과일련번호 : 0 ~ 9
# TEST_RST_SNO_SEQ 시험결과일련번호의항목순번 : 0 ~ 2

In [647]:
col12 = 'TEST_RST_SNO_SEQ'
print(cols_dict[name12][col12])
set(ct12[col12].unique())

시험결과일련번호의항목순번


{0, 1, 2}

## 과제2 > 건설기계 인증대장

In [126]:
name = '건설기계 인증대장(Tier3~Stage5)'
file_name = f'{name}.xlsx'
cert_ld34 = pd.read_excel(os.path.join(raw2_fold, file_name), sheet_name='TIER-3~TIER-4')
cert_ld34.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5974 entries, 0 to 5973
Data columns (total 64 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   배출기준             5971 non-null   object
 1   상호명              5971 non-null   object
 2   인증번호             5971 non-null   object
 3   최초인증일자           5971 non-null   object
 4   재교부일자
(변경일자)     5971 non-null   object
 5   접수번호             5971 non-null   object
 6   접수일자             5971 non-null   object
 7   종류               5971 non-null   object
 8   구분               5971 non-null   object
 9   상세내역             4051 non-null   object
 10  엔진형식
(엔진모델명)     5971 non-null   object
 11  엔진코드             5970 non-null   object
 12  엔진배기량            5862 non-null   object
 13  최대출력             5856 non-null   object
 14  Unnamed: 14      5856 non-null   object
 15  엔진제작사            5971 non-null   object
 16  엔진제작국            5971 non-null   object
 17  건설기계 
수입사
(대표자)  5367 non-null   

In [161]:
name = '건설기계 인증대장(Tier3~Stage5)'
file_name = f'{name}.xlsx'
cert_ld5 = pd.read_excel(os.path.join(raw2_fold, file_name), sheet_name='STAGE5')
cert_ld5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2058 entries, 0 to 2057
Data columns (total 74 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   배출기준             2054 non-null   object 
 1   상호명              2055 non-null   object 
 2   인증번호             2055 non-null   object 
 3   최초인증일자           2054 non-null   object 
 4   재교부일자
(변경일자)     2054 non-null   object 
 5   접수번호             2054 non-null   object 
 6   접수일자             2054 non-null   object 
 7   종류               2054 non-null   object 
 8   구분               2054 non-null   object 
 9   상세내역             783 non-null    object 
 10  엔진형식
(엔진모델명)     2054 non-null   object 
 11  엔진코드             2053 non-null   object 
 12  엔진배기량            2054 non-null   float64
 13  최대출력             2050 non-null   object 
 14  Unnamed: 14      2050 non-null   object 
 15  엔진
제작사           2054 non-null   object 
 16  엔진제작국            2054 non-null   object 
 17  건설기계 
수입사
(대표자

### 정보(TIER-3~TIER-4)

#### 컬럼 수정

In [128]:
cert_ld34.columns

Index(['배출기준', '상호명', '인증번호', '최초인증일자', '재교부일자\n(변경일자)', '접수번호', '접수일자', '종류',
       '구분', '상세내역', '엔진형식\n(엔진모델명)', '엔진코드', '엔진배기량', '최대출력', 'Unnamed: 14',
       '엔진제작사', '엔진제작국', '건설기계 \n수입사\n(대표자)', '건설기계\n제작사', '건설기계\n명칭',
       '건설/농업기계 형식', '비고', '정정내역', '기준치', 'Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', '시험결과', 'Unnamed: 30',
       'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42',
       'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46',
       'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50',
       'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54',
       'Unnamed: 55', '자체시험여부', '적용기술', 'Unnamed: 58', 'Unnamed: 59',
       'Unnamed: 60', 'Unnamed: 61', 'Unnamed: 62', 'DF 변경보고'],
      dtype='object')

In [136]:
cert_ld34.columns = [
    '배출기준', '상호명', '인증번호', '최초인증일자', '재교부일자(변경일자)', '접수번호', '접수일자', '종류', '구분', '상세내역', '엔진형식(엔진모델명)', '엔진코드', '엔진배기량', '최대출력_KW', '최대출력_RPM', '엔진제작사', '엔진제작국', '건설기계수입사(대표자)', '건설기계제작사', '건설기계명칭', '건설/농업기계_형식', '비고', '정정내역', 
    '기준치_CO', '기준치_NMHC+Nox', '기준치_NMHC', '기준치_Nox', '기준치_PM', '기준치_CO2', 
    '시험결과_ISO_8178_C1_CO', '시험결과_ISO_8178_C1_HC+Nox', '시험결과_ISO_8178_C1_NMHC', '시험결과_ISO_8178_C1_Nox', '시험결과_ISO_8178_C1_PM', '시험결과_ISO_8178_C1_CO2', '시험결과_ISO_8178_C1_검사일시', '시험결과_ISO_8178_C1_최대출력_kw', '시험결과_ISO_8178_C1_rpm', 
    '시험결과_NRSC_CO', '시험결과_NRSC_HC+Nox', '시험결과_NRSC_NMHC', '시험결과_NRSC_Nox', '시험결과_NRSC_PM', '시험결과_NRSC_CO2', '시험결과_NRSC_검사일시', '시험결과_NRSC_최대출력_kw', '시험결과_NRSC_최대출력_rpm', 
    '시험결과_NRTC_CO', '시험결과_NRTC_HC+Nox', '시험결과_NRTC_NMHC', '시험결과_NRTC_Nox', '시험결과_NRTC_PM', '시험결과_NRTC_CO2', '시험결과_NRTC_검사일시', '시험결과_NRTC_최대출력_kw', '시험결과_NRTC_최대출력_rpm',
    '자체시험여부', '적용기술_EGR', '적용기술_SCR', '적용기술_DPF', '적용기술_ECU', '적용기술_DOC', '적용기술_기타', 'DF_변경보고'
]
len(cert_ld34.columns)

64

#### 필요한 데이터만 추출(행 제한)

In [137]:
cert_ld34 = cert_ld34.iloc[3:, :]
cert_ld34.head()

,배출기준,상호명,인증번호,최초인증일자,재교부일자(변경일자),접수번호,접수일자,종류,구분,상세내역,...,시험결과_NRTC_최대출력_kw,시험결과_NRTC_최대출력_rpm,자체시험여부,적용기술_EGR,적용기술_SCR,적용기술_DPF,적용기술_ECU,적용기술_DOC,적용기술_기타,DF_변경보고
3,tier-3,"㈜커민스 코리아, 윤석윤",09EN*CU*01,2009-01-14 00:00:00,-,6940,2008-12-23 00:00:00,건설기계,대표(최초),NaN,...,-,-,"TIER-2기준으로시험,3도만족\n교통입회",X,X,X,O,NaN,NaN,해당없음
4,tier-3,"㈜커민스 코리아, 윤석윤",09EN*CU*01,-,2012-02-08 00:00:00,15,2012-02-06 00:00:00,건설기계,대표(정정),대표엔진의\n건설기계 형식 정정,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
5,tier-3,"㈜커민스 코리아, 윤석윤",09EN*CU*01,-,2011-08-03 00:00:00,173,2011-07-26 00:00:00,건설기계,대표(보고),대표엔진의\n수입사 및 건설기계추가,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
6,tier-3,"㈜커민스 코리아, 윤석윤",09EN*CU*01,-,2011-09-22 00:00:00,204,2011-09-07 00:00:00,건설기계,대표(보고),대표엔진의\n수입사 및 건설기계추가,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
7,tier-3,"㈜커민스 코리아, 윤석윤",09EN*CU*01,-,2012-01-25 00:00:00,3,2012-01-10 00:00:00,건설기계,대표(보고),대표엔진의\n수입사 및 건설기계추가,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음


In [184]:
cert_ld34.shape

(5971, 64)

### 데이터 확인

In [509]:
set(cert_ld34['DF_변경보고'].unique())

{'16년부터 수입 안함', 2200, 'O', nan, '신규인증완료후 미생산', '해당없음'}

### 데이터 수정(TIER-3~TIER-4)

In [ ]:
# 줄 바꿈이 있는 열
# 상세내역, 최대출력_KW, 최대출력_RPM, 정정내역, 자체시험여부

# 줄 바꿈을 간격으로 바꿀 열
# 상세내역

# 줄 바꿈을 없앨 열
# 최대출력_KW, 최대출력_RPM, 정정내역

# 줄 바꿈을 쉼표로 바꿀 열
# 자체시험여부

In [141]:
# 줄 바꿈을 간격으로 바꿀 열
# 상세내역
cert_ld34['상세내역'] = cert_ld34['상세내역'].str.replace('\n', ' ')
cert_ld34[['상세내역']].head()

,상세내역
3,NaN
4,대표엔진의 건설기계 형식 정정
5,대표엔진의 수입사 및 건설기계추가
6,대표엔진의 수입사 및 건설기계추가
7,대표엔진의 수입사 및 건설기계추가


In [140]:
# 줄 바꿈을 없앨 열
# 최대출력_KW, 최대출력_RPM, 정정내역
cert_ld34['최대출력_KW'] = cert_ld34['최대출력_KW'].str.replace('\n', '')
cert_ld34['최대출력_RPM'] = cert_ld34['최대출력_RPM'].str.replace('\n', '')
cert_ld34['정정내역'] = cert_ld34['정정내역'].str.replace('\n', '')
cert_ld34[['최대출력_KW', '최대출력_RPM', '정정내역']].head()

,최대출력_KW,최대출력_RPM,정정내역
3,250(325ps),"2,100(1,900)","1. 동일엔진 추가, 출력병기 (2010.02.02, 재교부)"
4,250(325ps),"2,100(1,900)","1.건설기계 형식 정정(2012.02.,재교부)"
5,250(325ps),"2,100(1,900)",NaN
6,250(325ps),"2,100(1,900)",NaN
7,250(325ps),"2,100(1,900)",NaN


In [142]:
# 줄 바꿈을 쉼표로 바꿀 열
# 자체시험여부
cert_ld34['자체시험여부'] = cert_ld34['자체시험여부'].str.replace('\n', ',')
cert_ld34[['자체시험여부']].head()

,자체시험여부
3,"TIER-2기준으로시험,3도만족,교통입회"
4,NaN
5,NaN
6,NaN
7,NaN


### 정보(STAGE5)

#### 컬럼 수정

In [162]:
cert_ld5.head()

,배출기준,상호명,인증번호,최초인증일자,재교부일자\n(변경일자),접수번호,접수일자,종류,구분,상세내역,...,Unnamed: 64,자체시험여부,DF 대표,적용기술,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,임의설정자료제출여부
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,EGR,SCR,DPF,ECU,DOC,기타,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,rpm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,건설기계,대표(최초),NaN,...,2000,자체시험,대표,-,O,-,O,O,NaN,X
4,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
cert_ld5.columns

Index(['배출기준', '상호명', '인증번호', '최초인증일자', '재교부일자\n(변경일자)', '접수번호', '접수일자', '종류',
       '구분', '상세내역', '엔진형식\n(엔진모델명)', '엔진코드', '엔진배기량', '최대출력', 'Unnamed: 14',
       '엔진\n제작사', '엔진제작국', '건설기계 \n수입사\n(대표자)', '건설기계\n제작사', '건설기계\n명칭',
       '건설/농업기계 형식', '비고', '기준치', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', '시험결과', 'Unnamed: 30',
       'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42',
       'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46',
       'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50',
       'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54',
       'Unnamed: 55', 'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58',
       'Unnamed: 59', 'Unnamed: 60', 'Unnamed: 61', 'Unnamed: 62',
       'Unnamed: 63', 'Unnamed: 64', '자체시험여부', 'DF 대표', '적용기술', 'Unnamed: 68',
 

In [164]:
cert_ld5.columns = [
    '배출기준', '상호명', '인증번호', '최초인증일자', '재교부일자(변경일자)', '접수번호', '접수일자', '종류', '구분', '상세내역', '엔진형식(엔진모델명)', '엔진코드', '엔진배기량', 
    '최대출력_KW', '최대출력_RPM',
    '엔진제작사', '엔진제작국', '건설기계수입사(대표자)', '건설기계제작사', '건설기계명칭', '건설/농업기계 형식', '비고', 
    '기준치_CO', '기준치_THC+Nox', '기준치_THC', '기준치_Nox', '기준치_PM', '기준치_PN', '기준치_NH3', 
    '시험결과_ISO_8178_C1_CO', '시험결과_ISO_8178_C1_THC+Nox', '시험결과_ISO_8178_C1_THC', '시험결과_ISO_8178_C1_Nox', '시험결과_ISO_8178_C1_PM', '시험결과_ISO_8178_C1_PN', '시험결과_ISO_8178_C1_NH3', '시험결과_ISO_8178_C1_CO2', '시험결과_ISO_8178_C1_연비', '시험결과_ISO_8178_C1_검사일시',
    '시험결과_ISO_8178_C1_최대출력_kw', '시험결과_ISO_8178_C1_최대출력_rpm', 
    '시험결과_NRSC_CO', '시험결과_NRSC_THC+Nox', '시험결과_NRSC_THC', '시험결과_NRSC_Nox', '시험결과_NRSC_PM', '시험결과_NRSC_PN', '시험결과_NRSC_NH3', '시험결과_NRSC_CO2', '시험결과_NRSC_연비', '시험결과_NRSC_검사일시', '시험결과_NRSC_최대출력_kw', '시험결과_NRSC_최대출력_rpm', 
    '시험결과_NRTC_CO', '시험결과_NRTC_THC+Nox', '시험결과_NRTC_THC', '시험결과_NRTC_Nox', '시험결과_NRTC_PM', '시험결과_NRTC_PN', '시험결과_NRTC_NH3', '시험결과_NRTC_CO2', '시험결과_NRTC_연비', '시험결과_NRTC_검사일시', '시험결과_NRTC_최대출력_kw', '시험결과_NRTC_최대출력_rpm', 
    '자체시험여부', 'DF_대표', '적용기술_EGR', '적용기술_SCR', '적용기술_DPF', '적용기술_ECU', '적용기술_DOC', '적용기술_기타', 
    '임의설정자료제출여부'
]

In [165]:
cert_ld5.head()

,배출기준,상호명,인증번호,최초인증일자,재교부일자(변경일자),접수번호,접수일자,종류,구분,상세내역,...,시험결과_NRTC_최대출력_rpm,자체시험여부,DF_대표,적용기술_EGR,적용기술_SCR,적용기술_DPF,적용기술_ECU,적용기술_DOC,적용기술_기타,임의설정자료제출여부
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,EGR,SCR,DPF,ECU,DOC,기타,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,rpm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,건설기계,대표(최초),NaN,...,2000,자체시험,대표,-,O,-,O,O,NaN,X
4,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 필요한 데이터만 추출(행 제한)

In [166]:
cert_ld5 = cert_ld5.iloc[3:, :]
cert_ld5.head()

,배출기준,상호명,인증번호,최초인증일자,재교부일자(변경일자),접수번호,접수일자,종류,구분,상세내역,...,시험결과_NRTC_최대출력_rpm,자체시험여부,DF_대표,적용기술_EGR,적용기술_SCR,적용기술_DPF,적용기술_ECU,적용기술_DOC,적용기술_기타,임의설정자료제출여부
3,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,건설기계,대표(최초),NaN,...,2000,자체시험,대표,-,O,-,O,O,NaN,X
4,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
cert_ld5.shape

(2055, 74)

In [167]:
cert_ld5.iloc[:5, :20]

,배출기준,상호명,인증번호,최초인증일자,재교부일자(변경일자),접수번호,접수일자,종류,구분,상세내역,엔진형식(엔진모델명),엔진코드,엔진배기량,최대출력_KW,최대출력_RPM,엔진제작사,엔진제작국,건설기계수입사(대표자),건설기계제작사,건설기계명칭
3,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,건설기계,대표(최초),NaN,F4HGE613I,F4HGE613I*V,6728.00,191,2000,FPT Industrial S.p.A,이태리,해당없음,해당없음,해당없음
4,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,F4HGE613I,F4DGE617D*V,6728.00,221,2100,FPT Industrial S.p.A,이태리,LS엠트론\n(구자은),NH,트랙터
5,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,F4HGE613I,F4DGE617D*V,6728.00,221,2100,FPT Industrial S.p.A,이태리,삼정건설기계\n(유재흥),CASE,트랙터
6,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,F4HGE613I,F4DGE617E*V,6728.00,199,2100,FPT Industrial S.p.A,이태리,LS엠트론\n(구자은),NH,트랙터
7,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,F4HGE613I,F4DGE617E*V,6728.00,199,2100,FPT Industrial S.p.A,이태리,삼정건설기계\n(유재흥),CASE,트랙터


In [168]:
cert_ld5.iloc[:5, 20:35]

,건설/농업기계 형식,비고,기준치_CO,기준치_THC+Nox,기준치_THC,기준치_Nox,기준치_PM,기준치_PN,기준치_NH3,시험결과_ISO_8178_C1_CO,시험결과_ISO_8178_C1_THC+Nox,시험결과_ISO_8178_C1_THC,시험결과_ISO_8178_C1_Nox,시험결과_ISO_8178_C1_PM,시험결과_ISO_8178_C1_PN
3,해당없음,NaN,3.50,-,0.19,0.40,0.01,1000000000000,10,-,-,-,-,-,-
4,T7.315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Optum 300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,T7.290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Optum 270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [169]:
cert_ld5.iloc[:5, 35:50]

,시험결과_ISO_8178_C1_NH3,시험결과_ISO_8178_C1_CO2,시험결과_ISO_8178_C1_연비,시험결과_ISO_8178_C1_검사일시,시험결과_ISO_8178_C1_최대출력_kw,시험결과_ISO_8178_C1_최대출력_rpm,시험결과_NRSC_CO,시험결과_NRSC_THC+Nox,시험결과_NRSC_THC,시험결과_NRSC_Nox,시험결과_NRSC_PM,시험결과_NRSC_PN,시험결과_NRSC_NH3,시험결과_NRSC_CO2,시험결과_NRSC_연비
3,-,-,-,-,-,-,0.01,-,0.01,0.29,0.00,34600000000,1.52,686.36,212.78
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
cert_ld5.iloc[:5, 50:60]

,시험결과_NRSC_검사일시,시험결과_NRSC_최대출력_kw,시험결과_NRSC_최대출력_rpm,시험결과_NRTC_CO,시험결과_NRTC_THC+Nox,시험결과_NRTC_THC,시험결과_NRTC_Nox,시험결과_NRTC_PM,시험결과_NRTC_PN,시험결과_NRTC_NH3
3,2019-11-14 00:00:00,191,2000,0.02,-,0.01,0.21,0.00,25100000000,0.76
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [171]:
cert_ld5.iloc[:5, 60:]

,시험결과_NRTC_CO2,시험결과_NRTC_연비,시험결과_NRTC_검사일시,시험결과_NRTC_최대출력_kw,시험결과_NRTC_최대출력_rpm,자체시험여부,DF_대표,적용기술_EGR,적용기술_SCR,적용기술_DPF,적용기술_ECU,적용기술_DOC,적용기술_기타,임의설정자료제출여부
3,706.07,219.15,2019-11-13 00:00:00,191,2000,자체시험,대표,-,O,-,O,O,NaN,X
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 데이터 확인

In [522]:
set(cert_ld5['비고'].unique())

{nan,
 '건설기계 명칭 정정재교부(22.9.8)',
 '건설기계 수입사 대표자 정정재교부(22.4.29)',
 '건설기계 제작사 대표자 정정재교부(22.11.08)',
 '건설기계 제작사 정정재교부(23.4.4)',
 '건설기계 제작사 정정재교부(23.5.3)',
 '건설기계 제작사명 변경 재교부(23.7.10)',
 '건설기계 형식 정정재교부(21.6.15,교통환경연구소-3376)\n건설기계 수입사 정정재교부(21.8.11, 교통환경연구소-4543)',
 '건설기계 형식 정정재교부(21.9.1,교통환경연구소-4961)',
 '건설기계 형식 정정재교부(22.10.27, 교통환경연구소-6606)',
 '건설기계 형식 정정재교부(22.12.21)',
 '건설기계 형식 정정재교부(22.6.30)',
 '건설기계 형식 정정재교부(22.7.13) / 제작사 정정재교부(23.4.00)',
 '건설기계 형식 정정재교부(22.7.15)',
 '건설기계형식 정정재교부(22.10.7, 교통환경연구소-6230)',
 '건설기계형식 정정재교부(22.3.23, 교통환경연구소-1737)',
 '농업기계형식 정정재교부(22.2.11, 교통환경연구소-1012)',
 '농업기계형식 정정재교부(22.2.11, 교통환경연구소-1012)\n엔진제작사 및 농업기게 제작사 정정재교부(22.10.27,교통환경연구소-6608)',
 '수입사 대표자 변경 재교부(22.10.14)',
 '수입사명 변경 재교부(23.7.10)',
 '엔진제작국 정정재교부(22.10.27, 교통환경연구소-6607)',
 '엔진제작사 정정재교부(23.9.22)',
 '엔진코드 정정재교부(21.3.25)',
 '재생지수 산정 결과 신규인증 결과 대입',
 '정정재교부(교통환경연구소-000,22.5.00)',
 '정정재교부(교통환경연구소-3899,21.7.12)',
 '제작사 정정재교부(23.4.00)',
 '제작사명 정정재교부(23.4.11)',
 '중고',
 '최대출력 정정재교부(22.9.8)',
 '최대출력 정정재교부(

### 데이터 수정(STAGE5)

In [172]:
# 줄 바꿈이 있는 열
# 건설기계수입사(대표자)

# 줄 바꿈을 없앨 열
# 건설기계수입사(대표자)

In [173]:
# 줄 바꿈을 없앨 열
# 건설기계수입사(대표자)
cert_ld5['건설기계수입사(대표자)'] = cert_ld5['건설기계수입사(대표자)'].str.replace('\n', '')
cert_ld5[['건설기계수입사(대표자)']].head()

,건설기계수입사(대표자)
3,해당없음
4,LS엠트론(구자은)
5,삼정건설기계(유재흥)
6,LS엠트론(구자은)
7,삼정건설기계(유재흥)


#### 컬럼, 데이터 정보들 출력

In [179]:
info01 = cert_ld34.isnull().sum().reset_index().rename(columns={'index':'컬럼명', 0:'빈 값 개수'})
info01['총 행수'] = cert_ld34.shape[0]
info01['번호'] = [x for x in range(1, info01.shape[0] + 1)]
info01 = info01[['번호', '컬럼명', '빈 값 개수', '총 행수']]
info01

,번호,컬럼명,빈 값 개수,총 행수
0,1,배출기준,0,5971
1,2,상호명,0,5971
2,3,인증번호,0,5971
3,4,최초인증일자,0,5971
4,5,재교부일자(변경일자),0,5971
...,...,...,...,...
59,60,적용기술_DPF,5683,5971
60,61,적용기술_ECU,5692,5971
61,62,적용기술_DOC,5771,5971
62,63,적용기술_기타,5882,5971


In [180]:
info02 = cert_ld5.isnull().sum().reset_index().rename(columns={'index':'컬럼명', 0:'빈 값 개수'})
info02['총 행수'] = cert_ld5.shape[0]
info02['번호'] = [x for x in range(1, info02.shape[0] + 1)]
info02 = info02[['번호', '컬럼명', '빈 값 개수', '총 행수']]
info02

,번호,컬럼명,빈 값 개수,총 행수
0,1,배출기준,1,2055
1,2,상호명,0,2055
2,3,인증번호,0,2055
3,4,최초인증일자,1,2055
4,5,재교부일자(변경일자),1,2055
...,...,...,...,...
69,70,적용기술_DPF,1925,2055
70,71,적용기술_ECU,1925,2055
71,72,적용기술_DOC,1925,2055
72,73,적용기술_기타,1951,2055


In [182]:
today_date = datetime.today().strftime('%Y.%m.%d')
today_date

'2023.10.31'

In [183]:
with pd.ExcelWriter(os.path.join(an2_fold, f'건설기계_인증대장_데이터정보({today_date}).xlsx')) as writer:
    info01.to_excel(writer, sheet_name='TIER-3~TIER-4', index=False)
    info02.to_excel(writer, sheet_name='STAGE5', index=False)

## 전문정비(STD_TB_ERR_RES)

In [558]:
er_dict = {
    'DRVNG_DSTNC':'주행거리', 
    'VHRNO':'자동차등록번호', 
    'VIN':'차대번호', 
    'LOD_COX_MEVLU':'부하일산화탄소측정치', 
    'LOD_HC_MEVLU':'부하탄화수소측정치', 
    'LOD_NOX_MEVLU':'부하질소산화물측정치', 
    'LOD_SMO_MEVLU':'부하매연측정치', 
    'LOD_TQ_MEVLU':'부하토크측정치', 
    'OVHUL_YMD':'정비일자', 
    'SPCY_OVHUL_BIZPLC_NM':'전문정비사업소명', 
    'OVHUL_RCPT_NO':'정비접수번호', 
    'PRCINSP_RINSP_RCPT_NO':'정밀재검사접수번호', 
    'NOLOD_COX_MEVLU':'무부하일산화탄소측정치', 
    'NOLOD_HC_MEVLU':'무부하탄화수소측정치', 
    'NOLOD_AIR_EXCRT_MEVLU':'무부하공기과잉률측정치', 
    'NOLOD_SMO_MEVLU':'무부하매연측정치', 
}

In [559]:
# 0.3s
name = 'STD_TB_ERR_RES(20231011)'
file_name = f'{name}.csv'
er = pd.read_csv(os.path.join(pub_fold, file_name), low_memory=False)
er.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54617 entries, 0 to 54616
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DRVNG_DSTNC            54617 non-null  float64
 1   VHRNO                  54617 non-null  object 
 2   VIN                    51051 non-null  object 
 3   LOD_COX_MEVLU          53884 non-null  object 
 4   LOD_HC_MEVLU           53886 non-null  object 
 5   LOD_NOX_MEVLU          53883 non-null  object 
 6   LOD_SMO_MEVLU          53941 non-null  object 
 7   LOD_TQ_MEVLU           53819 non-null  object 
 8   OVHUL_YMD              54617 non-null  int64  
 9   SPCY_OVHUL_BIZPLC_NM   54617 non-null  object 
 10  OVHUL_RCPT_NO          54617 non-null  int64  
 11  PRCINSP_RINSP_RCPT_NO  54617 non-null  object 
 12  NOLOD_COX_MEVLU        53870 non-null  object 
 13  NOLOD_HC_MEVLU         53871 non-null  object 
 14  NOLOD_AIR_EXCRT_MEVLU  28484 non-null  object 
 15  NO

In [560]:
er = er.rename(columns=er_dict)
er.columns

Index(['주행거리', '자동차등록번호', '차대번호', '부하일산화탄소측정치', '부하탄화수소측정치', '부하질소산화물측정치',
       '부하매연측정치', '부하토크측정치', '정비일자', '전문정비사업소명', '정비접수번호', '정밀재검사접수번호',
       '무부하일산화탄소측정치', '무부하탄화수소측정치', '무부하공기과잉률측정치', '무부하매연측정치'],
      dtype='object')

### 데이터 확인

In [575]:
set(er['정비접수번호'].unique())

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13}

#### 컬럼, 데이터 정보들 출력

In [562]:
er_cols = pd.DataFrame({'컬럼명':er_dict.keys(), '컬럼내용':er_dict.values()})
er_cols

,컬럼명,컬럼내용
0,DRVNG_DSTNC,주행거리
1,VHRNO,자동차등록번호
2,VIN,차대번호
3,LOD_COX_MEVLU,부하일산화탄소측정치
4,LOD_HC_MEVLU,부하탄화수소측정치
5,LOD_NOX_MEVLU,부하질소산화물측정치
6,LOD_SMO_MEVLU,부하매연측정치
7,LOD_TQ_MEVLU,부하토크측정치
8,OVHUL_YMD,정비일자
9,SPCY_OVHUL_BIZPLC_NM,전문정비사업소명


In [566]:
temp = er.isnull().sum().reset_index()
temp = temp.rename(columns={'index':'컬럼내용', 0:'빈 값 개수'})
temp['총 행수'] = er.shape[0]
temp['번호'] = [x for x in range(1, temp.shape[0] + 1)]
ttemp = temp.merge(er_cols, on='컬럼내용', how='left')
ttemp = ttemp[['번호', '컬럼명', '컬럼내용', '빈 값 개수', '총 행수']]
ttemp

,번호,컬럼명,컬럼내용,빈 값 개수,총 행수
0,1,DRVNG_DSTNC,주행거리,0,54617
1,2,VHRNO,자동차등록번호,0,54617
2,3,VIN,차대번호,3566,54617
3,4,LOD_COX_MEVLU,부하일산화탄소측정치,733,54617
4,5,LOD_HC_MEVLU,부하탄화수소측정치,731,54617
5,6,LOD_NOX_MEVLU,부하질소산화물측정치,734,54617
6,7,LOD_SMO_MEVLU,부하매연측정치,676,54617
7,8,LOD_TQ_MEVLU,부하토크측정치,798,54617
8,9,OVHUL_YMD,정비일자,0,54617
9,10,SPCY_OVHUL_BIZPLC_NM,전문정비사업소명,0,54617


In [567]:
today_date = datetime.today().strftime('%Y.%m.%d')
today_date

'2023.10.31'

In [568]:
ttemp.to_excel(os.path.join(an4_fold, f'전문정비_데이터정보({today_date}).xlsx'), index=False)

## 전문정비상세(STD_TB_ERR_RES_DET)

In [554]:
erd_dict = {
    'VHRNO':'자동차등록번호', 
    'VIN':'차대번호', 
    'OVHUL_CN':'정비내용', 
    'OVHUL_YMD':'정비일자', 
    'SPCY_OVHUL_BIZPLC_NM':'전문정비사업소명', 
    'OVHUL_CMPNT_NM':'정비부품명', 
    'OVHUL_RCPT_NO':'정비접수번호', 
    'PRCINSP_RINSP_RCPT_NO':'정밀재검사접수번호', 
}

In [555]:
# 0.3s
name = 'STD_TB_ERR_RES_DET(20231011)'
file_name = f'{name}.csv'
erd = pd.read_csv(os.path.join(pub_fold, file_name), low_memory=False)
erd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106819 entries, 0 to 106818
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VHRNO                  106818 non-null  object 
 1   VIN                    94670 non-null   object 
 2   OVHUL_CN               106817 non-null  object 
 3   OVHUL_YMD              106818 non-null  float64
 4   SPCY_OVHUL_BIZPLC_NM   106818 non-null  object 
 5   OVHUL_CMPNT_NM         106817 non-null  object 
 6   OVHUL_RCPT_NO          106818 non-null  float64
 7   PRCINSP_RINSP_RCPT_NO  0 non-null       float64
dtypes: float64(3), object(5)
memory usage: 6.5+ MB


In [556]:
erd = erd.rename(columns=erd_dict)
erd.columns

Index(['자동차등록번호', '차대번호', '정비내용', '정비일자', '전문정비사업소명', '정비부품명', '정비접수번호',
       '정밀재검사접수번호'],
      dtype='object')

### 데이터 확인

In [557]:
erd.shape

(106819, 8)

In [581]:
set(erd['정비접수번호'].unique())

{nan}

#### 컬럼, 데이터 정보들 출력

In [569]:
erd_cols = pd.DataFrame({'컬럼명':erd_dict.keys(), '컬럼내용':erd_dict.values()})
erd_cols

,컬럼명,컬럼내용
0,VHRNO,자동차등록번호
1,VIN,차대번호
2,OVHUL_CN,정비내용
3,OVHUL_YMD,정비일자
4,SPCY_OVHUL_BIZPLC_NM,전문정비사업소명
5,OVHUL_CMPNT_NM,정비부품명
6,OVHUL_RCPT_NO,정비접수번호
7,PRCINSP_RINSP_RCPT_NO,정밀재검사접수번호


In [570]:
temp = erd.isnull().sum().reset_index()
temp = temp.rename(columns={'index':'컬럼내용', 0:'빈 값 개수'})
temp['총 행수'] = erd.shape[0]
temp['번호'] = [x for x in range(1, temp.shape[0] + 1)]
ttemp = temp.merge(erd_cols, on='컬럼내용', how='left')
ttemp = ttemp[['번호', '컬럼명', '컬럼내용', '빈 값 개수', '총 행수']]
ttemp

,번호,컬럼명,컬럼내용,빈 값 개수,총 행수
0,1,VHRNO,자동차등록번호,1,106819
1,2,VIN,차대번호,12149,106819
2,3,OVHUL_CN,정비내용,2,106819
3,4,OVHUL_YMD,정비일자,1,106819
4,5,SPCY_OVHUL_BIZPLC_NM,전문정비사업소명,1,106819
5,6,OVHUL_CMPNT_NM,정비부품명,2,106819
6,7,OVHUL_RCPT_NO,정비접수번호,1,106819
7,8,PRCINSP_RINSP_RCPT_NO,정밀재검사접수번호,106819,106819


In [572]:
today_date = datetime.today().strftime('%Y.%m.%d')
today_date

'2023.10.31'

In [571]:
ttemp.to_excel(os.path.join(an4_fold, f'전문정비상세_데이터정보({today_date}).xlsx'), index=False)

## 구조변경검사(STD_DLM_TB_ERP_GUBUN_BAECHUL)

In [525]:
# 11.7s
name = 'STD_DLM_TB_ERP_GUBUN_BAECHUL(20231011)'
file_name = f'{name}.csv'
gb = pd.read_csv(os.path.join(pub_fold, file_name), low_memory=False)
gb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1321945 entries, 0 to 1321944
Data columns (total 39 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   AIR_PSSR                   1152822 non-null  float64
 1   AIR_TMPRT                  1152822 non-null  float64
 2   CO2_VAL                    1110507 non-null  float64
 3   DRVNG_DSTNC                1320563 non-null  float64
 4   FNCINSP_JGMT_YN            1321919 non-null  object 
 5   INSPOF_NM                  1321945 non-null  object 
 6   INSPOF_RPRSV_NM            1318198 non-null  object 
 7   MAXM_OTPT_MSRMT_VAL        445619 non-null   float64
 8   MAXM_OTPT_PRMT_VAL         544747 non-null   float64
 9   O2_VAL                     1110507 non-null  float64
 10  PRCINSP_YMD                1321945 non-null  int64  
 11  PRCINSP_VLD_YMD            127529 non-null   float64
 12  PRCINSP_VLD_WHL            1321945 non-null  object 
 13  PRCINSP_JGMT

In [ ]:
gb.shape

(1321945, 39)

In [ ]:
gb.columns

Index(['AIR_PSSR', 'AIR_TMPRT', 'CO2_VAL', 'DRVNG_DSTNC', 'FNCINSP_JGMT_YN',
       'INSPOF_NM', 'INSPOF_RPRSV_NM', 'MAXM_OTPT_MSRMT_VAL',
       'MAXM_OTPT_PRMT_VAL', 'O2_VAL', 'PRCINSP_YMD', 'PRCINSP_VLD_YMD',
       'PRCINSP_VLD_WHL', 'PRCINSP_JGMT', 'PRCINSP_KND', 'PRCINSP_RCPT_NO',
       'RINSP_WHL', 'VHCL_PURPS_CD', 'VHRNO', 'VIN',
       'STRCT_CHG_INSP_MSRMT_MTHD', 'NOLOD_SMO_MSRMT_VAL1',
       'NOLOD_SMO_PRMT_VAL1', 'NOLOD_SMO_MSRMT_VAL2', 'NOLOD_SMO_PRMT_VAL2',
       'NOLOD_SMO_MSRMT_VAL3', 'NOLOD_SMO_PRMT_VAL3', 'NOLOD_SMO_MSRMT_VAL4',
       'NOLOD_SMO_PRMT_VAL4', 'NOLOD_SMO_MSRMT_VAL5', 'NOLOD_SMO_PRMT_VAL5',
       'NOLOD_SMO_MSRMT_VAL6', 'NOLOD_SMO_PRMT_VAL6', 'NOLOD_SMO_JGMT_YN1',
       'NOLOD_SMO_JGMT_YN2', 'NOLOD_SMO_JGMT_YN3', 'NOLOD_SMO_JGMT_YN4',
       'NOLOD_SMO_JGMT_YN5', 'NOLOD_SMO_JGMT_YN6'],
      dtype='object')

### 데이터 확인

In [ ]:
gb.iloc[:, :10].head()

,AIR_PSSR,AIR_TMPRT,CO2_VAL,DRVNG_DSTNC,FNCINSP_JGMT_YN,INSPOF_NM,INSPOF_RPRSV_NM,MAXM_OTPT_MSRMT_VAL,MAXM_OTPT_PRMT_VAL,O2_VAL
0,0.00,0.00,14.80,25377.00,Y,이현자동차검사소,최원수,NaN,NaN,0.00
1,101.30,18.00,0.00,172542.00,Y,서인천자동차검사소,박상영,NaN,NaN,0.00
2,98.20,24.80,0.00,203176.00,Y,용인자동차검사소,최광진,NaN,NaN,0.00
3,101.20,15.20,0.00,208787.00,Y,노원자동차검사소,김동연,NaN,NaN,0.00
4,99.80,13.40,0.00,158139.00,Y,성동자동차검사소,나성철,NaN,NaN,0.00


In [ ]:
set(gb.FNCINSP_JGMT_YN.unique())

{' ', '8', 'N', 'Y', nan}

In [ ]:
gb.iloc[:, 10:20].head()

,PRCINSP_YMD,PRCINSP_VLD_YMD,PRCINSP_VLD_WHL,PRCINSP_JGMT,PRCINSP_KND,PRCINSP_RCPT_NO,RINSP_WHL,VHCL_PURPS_CD,VHRNO,VIN
0,20121031,NaN,~,Y,A21C02,27302012103101971,20121031~20121109,A20P,43버6629,KMHFG41EBCA120332
1,20121031,NaN,~,Y,A21C02,28402012103101621,20121031~20121109,A20P,81노1182,KMFLA18EP6C009216
2,20121031,NaN,~,Y,A21C02,41j02012103101191,20121031~20121109,A20P,경기94다4315,KN3HDP3352K836554
3,20121031,NaN,~,Y,A21C02,11602012103102571,20121031~20121109,A20P,서울34머6188,KMHWP81HP2U498130
4,20121031,NaN,~,Y,A21C02,11a02012103102091,20121031~20121109,A20P,80조2956,KMFWVH7HP5U646771


In [ ]:
set(gb.VHCL_PURPS_CD.unique())

{'A20B', 'A20O', 'A20P', 'A20T'}

In [ ]:
gb.iloc[:, 20:30].head()

,STRCT_CHG_INSP_MSRMT_MTHD,NOLOD_SMO_MSRMT_VAL1,NOLOD_SMO_PRMT_VAL1,NOLOD_SMO_MSRMT_VAL2,NOLOD_SMO_PRMT_VAL2,NOLOD_SMO_MSRMT_VAL3,NOLOD_SMO_PRMT_VAL3,NOLOD_SMO_MSRMT_VAL4,NOLOD_SMO_PRMT_VAL4,NOLOD_SMO_MSRMT_VAL5
0,부하검사(ASM-Idling),0.00,0.60,7.00,120.00,0.00,790.00,0.00,1.00,8.00
1,부하검사(LUG DOWN),0.00,20.00,0.00,20.00,0.00,20.00,2504.00,5.00,123.80
2,부하검사(KD-147),14.00,20.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,부하검사(KD-147),1.00,20.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,부하검사(KD-147),0.00,20.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
set(gb.STRCT_CHG_INSP_MSRMT_MTHD.unique())

{'무부하검사(TSI)',
 '무부하검사(급가속)',
 '무부하검사(정지가동)',
 '부하검사(ASM-Idling)',
 '부하검사(ASM2525)',
 '부하검사(KD-147)',
 '부하검사(LUG DOWN)'}

In [ ]:
gb.iloc[:, 30:40].head()

,NOLOD_SMO_PRMT_VAL5,NOLOD_SMO_MSRMT_VAL6,NOLOD_SMO_PRMT_VAL6,NOLOD_SMO_JGMT_YN1,NOLOD_SMO_JGMT_YN2,NOLOD_SMO_JGMT_YN3,NOLOD_SMO_JGMT_YN4,NOLOD_SMO_JGMT_YN5,NOLOD_SMO_JGMT_YN6
0,120.00,0.98,0.10,Y,Y,Y,Y,Y,Y
1,117.00,NaN,NaN,Y,Y,Y,Y,Y,NaN
2,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN


In [ ]:
set(gb.NOLOD_SMO_JGMT_YN6.unique())

{' ', '0', 'N', 'Y', nan}

#### 컬럼, 데이터 정보들 출력

In [ ]:
temp = gb.isnull().sum().reset_index()
temp = temp.rename(columns={'index':'컬럼명', 0:'빈 값 개수'})
temp['총 행수'] = gb.shape[0]
temp['번호'] = [x for x in range(1, temp.shape[0] + 1)]
temp = temp[['번호', '컬럼명', '빈 값 개수', '총 행수']]
temp

,번호,컬럼명,빈 값 개수,총 행수
0,1,AIR_PSSR,169123,1321945
1,2,AIR_TMPRT,169123,1321945
2,3,CO2_VAL,211438,1321945
3,4,DRVNG_DSTNC,1382,1321945
4,5,FNCINSP_JGMT_YN,26,1321945
5,6,INSPOF_NM,0,1321945
6,7,INSPOF_RPRSV_NM,3747,1321945
7,8,MAXM_OTPT_MSRMT_VAL,876326,1321945
8,9,MAXM_OTPT_PRMT_VAL,777198,1321945
9,10,O2_VAL,211438,1321945


In [ ]:
today_date = datetime.today().strftime('%Y.%m.%d')
today_date

In [ ]:
temp.to_excel(os.path.join(an5_fold, f'구조변경_데이터정보({today_date}).xlsx'), index=False)

# EDA

## 전문정비(STD_TB_ERR_RES)

In [17]:
er['정비일자'].isnull().sum()

0

In [15]:
er['정비일자'].describe()

count      54617.00
mean    20083896.95
std        16435.80
min     20060727.00
25%     20070801.00
50%     20080507.00
75%     20091109.00
max     20121010.00
Name: 정비일자, dtype: float64

In [19]:
er['정비일자'] = pd.to_datetime(er['정비일자'], format='%Y%m%d', errors='coerce')
er['정비일자'].head()

0   2008-01-10
1   2008-01-23
2   2008-01-08
3   2008-01-18
4   2008-01-07
Name: 정비일자, dtype: datetime64[ns]

In [20]:
er['정비일자'].describe()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_3124\328505498.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  er['정비일자'].describe()


count                   54617
unique                   1839
top       2007-10-02 00:00:00
freq                      123
first     2006-07-27 00:00:00
last      2012-10-10 00:00:00
Name: 정비일자, dtype: object

## 전문정비상세(STD_TB_ERR_RES_DET)

In [18]:
erd['정비일자'].isnull().sum()

1

In [16]:
erd['정비일자'].describe()

count     106818.00
mean    20083911.85
std        15852.89
min     20060727.00
25%     20070829.00
50%     20080529.00
75%     20090921.00
max     20120928.00
Name: 정비일자, dtype: float64

In [21]:
erd['정비일자'] = pd.to_datetime(erd['정비일자'], format='%Y%m%d', errors='coerce')
erd['정비일자'].head()

0   2007-02-08
1   2006-08-11
2   2006-08-11
3   2006-08-11
4   2006-12-11
Name: 정비일자, dtype: datetime64[ns]

In [22]:
erd['정비일자'].describe()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_3124\338169835.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  erd['정비일자'].describe()


count                  106818
unique                   1832
top       2007-08-13 00:00:00
freq                      232
first     2006-07-27 00:00:00
last      2012-09-28 00:00:00
Name: 정비일자, dtype: object

## 구조변경검사(STD_DLM_TB_ERP_GUBUN_BAECHUL)